In [1]:
import numpy as np
import pandas as pd

In [2]:
countries = ["D1", "E0", "F1", "I1", "SP1"]
years = range(2013, 2020)
providers = ['B365','BW','IW', 'PS', 'WH', 'VC']

In [3]:
def compute(country, year, provider):
    colh = "{}H".format(provider)
    cold = "{}D".format(provider)
    cola = "{}A".format(provider)

    data = pd.read_csv("../../data/Dataset2/{}_{}.csv".format(country, year))
    if colh not in data.columns:
        return

    data = data[["HomeTeam", "AwayTeam", "FTR", colh, cold, cola]]
    
    team_results = {}

    def find_results(row):
        #global team_results
        home = row["HomeTeam"]
        away = row["AwayTeam"]
        res = row["FTR"]
        oddsH = row[colh]
        oddsD = row[cold]
        oddsA = row[cola]

        if not home in team_results:
            team_results[home] = 0

        if not away in team_results:
            team_results[away] = 0

        pred = np.array([oddsH, oddsD, oddsA]).argmin()

        pred = np.array(["H", "D", "A"])[pred]

        if res == pred:
            team_results[home] += 1
            team_results[away] += 1
        return

    data.apply(find_results, axis = 1)
    total = (len(team_results) - 1) * 2
    team_results = pd.DataFrame.from_dict(team_results, orient="index", columns=["correct_pred"])

    team_results["wrong_pred"]=total-team_results["correct_pred"]
    
    team_results = team_results.sort_values("correct_pred")

    team_results = team_results.reset_index()

    team_results = team_results.rename(columns={"index": "name"})
    
    team_results.to_csv("data/{}_{}_{}.csv".format(country, year, provider.lower()), index=False)

In [4]:
for country in countries:
    for year in years:
        for provider in providers:
            compute(country, str(year), provider)